# Widgets

If you are like a lot of data scientists, you are using `Jupyter` to manage and code your experiments and analyses. Instead of presenting static content generated by your notebooks, you can also make your notebooks interactive by using [Jupyter Widgets](https://ipywidgets.readthedocs.io/en/stable/index.html). Let's dive into widgets and see how they work.

In [1]:
from ipywidgets import interact

def f(x):
    return x

interact(f, x=10);

interactive(children=(IntSlider(value=10, description='x', max=30, min=-10), Output()), _dom_classes=('widget-…